# Dynamic Tests

This jupyter notebook allows you to treat a video so that in its upper left corner appears the name of the label that the neural network thinks you are seeing at that moment.



In [11]:
# OS imports
import glob
import os,shutil
import re
# Keras imports
from keras.models import load_model
from keras.preprocessing import image as preprocessing
# Image processing imports
from PIL import Image, ImageFont, ImageDraw, ImageOps
import numpy as np
import cv2
# Other
import time

# My directories paths
commonPath = ""
videos = commonPath + "Dynamic Tests/Originals"
saveInDir = commonPath + "Dynamic Tests/Results"

In [12]:
mapVideo=[]
numTotalFrames = 4282
for i in range(0, 385):
    mapVideo.append(0)
for i in range(385, 640):
    mapVideo.append(1)
for i in range(640, 1260):
    mapVideo.append(2)
for i in range(1260, 2950):
    mapVideo.append(4)
for i in range(2950, 3600):
    mapVideo.append(5)
for i in range(3600, 3980):
    mapVideo.append(6)
for i in range(3980, numTotalFrames):
    mapVideo.append(3)

In [13]:
model = load_model(commonPath + 'new.h5')
#neuralNetworkImageSize = (96, 54)
neuralNetworkImageSize = (128, 72)

In [14]:
def lookForBest(prediction):
    best = 0
    categories = 6
    for i,probability in enumerate(prediction[0]):
        if probability > prediction[0][best]:
            best = i
    return best

def predictImg(img,realLabel,model):
    prediction = model.predict(img)
    return lookForBest(prediction)

def translateNode(node):
    if node == 0:
        return "Hall - Resident Rooms"
    if node == 1:
        return "Hall - Stairs"
    if node == 2:
        return "Building - Dinning room"
    if node == 3:
        return "Building - Rooms"
    if node == 4:
        return "Parking"
    if node == 5:
        return "Building - Library"
    if node == 6:
        return "Building - Gym"
    return "Unknown"

def printAndRemove(msg):
    print(msg, end="\r")
    time.sleep(0)

In [15]:
def testVideo(videoPath, realLabelsMap):
    
    videoName = videoPath.split('/')[2]
    cap = cv2.VideoCapture(videoPath)
    
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    newVideoFramesSize = (int(frameWidth/5), int(frameHeight/5))
    
    try:
        os.stat('.temp')
    except:
        os.mkdir('.temp')
    
    totalTrues = 0
    frameNumber = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            printAndRemove("Working on frame nº" + str(frameNumber))
            
            # Preprocessing of the image for the Neural Network
            img = generateNeuralNetworkImage(frame, neuralNetworkImageSize)
            # Use Neural Network to predict the node
            labelPredicted = translateNode(lookForBest(model.predict(img)))
            # We update the accuracy
            if(translateNode(realLabelsMap[frameNumber]) == labelPredicted or realLabelsMap[frameNumber] == -1):
                totalTrues = totalTrues + 1
            # Put label in node
            newFrame = generateNewVideoFrame(
                frame, 
                newVideoFramesSize,
                labelPredicted, 
                translateNode(realLabelsMap[frameNumber]), 
                totalTrues/(frameNumber+1)
            )
            
            # Save new frame in temporal directory
            cv2.imwrite(".temp/frame" + str(frameNumber) + ".jpg", cv2.cvtColor(newFrame, cv2.COLOR_BGR2RGB))
            frameNumber = frameNumber + 1
        else:
            break
    print("")
    # New Video with the new frames
    createNewVideoWithFrames(videoName, ".temp/*.jpg", newVideoFramesSize)
    
    shutil.move(videoName, saveInDir)
    cap.release()
    cv2.destroyAllWindows()
    #shutil.rmtree('.temp/')
        
    # Print results
    print("Video \"" + videoName + "\" ready")
    print("\t-True Positives: " + str(totalTrues))
    print("\t-Total Number Frames: " + str(frameNumber))
    print("\t-Accuracy: " + str(totalTrues/frameNumber))

def generateNeuralNetworkImage(image, size):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img).convert('RGB')
    img = img.rotate(180)
    img.thumbnail(neuralNetworkImageSize, Image.ANTIALIAS)
    img = np.rot90(img)
    img = Image.fromarray(img)
    img = np.expand_dims(img, axis=0)
    return img
    
def generateNewVideoFrame(frame, size, labelPredicted, labelReal, accuracy, ):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame).convert('RGB')
    frame = frame.rotate(180)
    frame.thumbnail(size, Image.ANTIALIAS)
    frameDraw = ImageDraw.Draw(frame)
    frameDraw.text( (5, 5) , "Prediction:   " + labelPredicted,  font = ImageFont.truetype("arial.ttf", 12), fill=(255,255,255))
    frameDraw.text( (5, 20) , "Real Label: " + labelReal,  font = ImageFont.truetype("arial.ttf", 12), fill=(255,255,255))
    frameDraw.text( (5, 35) , "Accuracy: " + str(accuracy),  font = ImageFont.truetype("arial.ttf", 12), fill=(255,255,255))
    frame = np.array(frame)
    return frame

def createNewVideoWithFrames(newVideoName, dirFrames, videoSize):
    video = cv2.VideoWriter(newVideoName, cv2.VideoWriter_fourcc(*'DIVX'), 30.0, videoSize)
    filenames = glob.glob(dirFrames)
    filenames.sort(key=recompile)

    for i,image in enumerate(filenames):
        printAndRemove("New Video - Working on frame nº" + str(i))
        video.write(cv2.imread(image))
    video.release()
    print("")
    
def recompile(name):
    r= re.compile("(\d+).*").split(name)
    return [int(y) if y.isdigit() else y for y in r]

In [17]:
testVideo(videos + "/test1.mp4", mapVideo)

Working on frame nº4281
New Video - Working on frame nº4281
Video "test1.mp4" ready
	-True Positives: 2233
	-Total Number Frames: 4282
	-Accuracy: 0.521485287248949
